Parameters

In [11]:
import numpy as np
from scipy.stats import norm

Annunity

In [2]:
#This method returns the Annunity of the installment
def Anu(PV, r, n, m):

        val = 1 - (1 / ((1 + (r/m))**(n)))

        A = (PV*(r/m))/val

        return A

In [3]:
#This method returns the duration of the annunity

def Duration_Anu(A, PV, r, maturity,m):

        sum = 0

        for i in range(1, maturity+1):
            sum = sum + ((A * (i/m)) / ((1 + (r/m))**i))

        dur = sum / PV
        return dur


Bonds

In [4]:
#This method finds the market price of the bond

def marketPrice(F, C, maturity, r, m): #m is the interval
        
        sum =0
        for i in range(1, maturity+1):
            sum = sum + (1 / (1+(r/m))**i)
        
        price = sum*(F*(C/m)) + F*(1/ (1+(r/m))**maturity)

        return price

In [5]:
#This method returns the duration of the bond
def duration(F,C,maturity,r,m):
        
        sum =0

        for i in range(1, maturity+1):
            sum = sum + (((C/m)*F*(i/m)) / ((1 + (r/m))**i))
        
        dur = (sum + ((F*maturity) / ((1 + (r/m)))**maturity))/ marketPrice(F,C,maturity,r,m)

        return dur /m

In [6]:
#This method solves the immunization stategy 

#if the question asks without shortselling then you choose one bond with a
#duration that is smaller than the obligation duration and one with a longer duration
def Immunization(dur1, dur2, dur3, m1, m2, m3, obligation, obligation_dur):

    coef = np.array([[m1, m2, m3],[m1*dur1, m2*dur2, m3*dur3]])
    val = np.array([obligation, obligation*obligation_dur])

    return np.linalg.lstsq(coef, val, rcond=None)[0]

result_Immu = Immunization(dur1=2.85735, dur2=15, 
                   dur3=0, m1=97.327,
                   m2=41.7265, m3=0,
                   obligation=4212.36, 
                   obligation_dur=2.88363)
print(result_Immu)

[43.18681667  0.2184869   0.        ]


Mininimum Variance and Market Portfolio (Tangency Portfolio)

In [7]:
#Tangency Portfolio

def marketPortfolio(mu, rt, cov):
    invCov = np.linalg.inv(cov)

    ones = np.array([1,1,1]) #the size of the array depends on the size of the covariance matrix
    weights = (mu - rt*ones).T @ invCov / ((mu - rt*ones).T @ invCov @ ones)

    mean = float(weights@mu)

    variance = float(weights @ cov @ weights.T)

    return mean, variance, weights

print(marketPortfolio(mu=np.array([1, 1/5, 1/2]), rt= 20/100, 
                      cov=np.array([[1/100, -1/200, 0], [-1/200, 1/4, 0],
                                                                            [0,0,16/25]])))

(0.9815749890519982, 0.009428722093321644, array([0.97484816, 0.01949696, 0.00565488]))


Minimum Variance Portfolio

In [8]:
def minimumPortfolio(mean,cov):

    invCov = np.linalg.inv(cov)

    transpose_1 = np.ones((1,3))
    w = (transpose_1@ invCov) / (transpose_1 @ invCov @transpose_1.T)

    mean = mean.T

    expected_return = float(w @ mean)
    variance = float(w @cov@w.T)

    return expected_return, variance, w

print(minimumPortfolio(mean=np.array([1, 1/5, 1/2]), 
                       cov= np.array([[1/100, -1/200, 0], [-1/200, 1/4, 0],[0,0,16/25]])))

(0.9491228070175438, 0.009037227214377405, array([[0.93110826, 0.05477107, 0.01412067]]))


In [10]:
#The covariance between the market and tangency portfolio. I did not understand the reason of this method
def covariance(weight_Market, weight_Min, Cov_Matrix):

    cov = weight_Market@Cov_Matrix@weight_Min.T

    return cov

print(covariance(marketPortfolio(np.array([1, 1/5, 1/2]), 20/100, np.array([[1/100, -1/200, 0],[-1/200,1/4,0],[0,0,16/25]])
                      )[2],minimumPortfolio(np.array([1, 1/5, 1/2]), np.array([[1/100, -1/200, 0],[-1/200,1/4,0],[0,0,16/25]]))[2]
                      ,np.array([[1/100, -1/200, 0],[-1/200,1/4,0],[0,0,16/25]]))[0])



0.009037227214377407


Black - Scholes

In [53]:
def call(var, r, K, S0, T, t):

    sigma = np.sqrt(var)
    drift = r - (1/2)*var

    mu = r

    d_1 = (1 / (sigma*np.sqrt(T-t)))*(np.log(S0/K) + ((r + (var /2))*(T-t)))

    d_2 = d_1 - (sigma*np.sqrt(T -t))

    #find the z scores of d_1 and d_2

    z_score_d1 = norm.cdf(d_1) 
    z_score_d2 = norm.cdf(d_2)

    c = (z_score_d1*S0) - ((z_score_d2*K)*np.exp(-r*(T-t)))

    return c

call_result = call(1/4, ((20/100)/52), 34, 35, 2/52, 0)

def put(call, K, T, t, S0, r):

    p = K*(np.exp(-r*(T-t))) - S0 + call

    return p

put_result = put(call_result, 34,2/52,0,35,(20/100)/52)


print(call_result,put_result)


1.9105582254606794 0.9055290116542984
